In [ ]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem
# suppress rdkit warning
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [ ]:
def group_f1_b(df): 
    label = np.array(df.Class_b.tolist(), dtype=int)
    pred = np.array(df.Pred_b.tolist(), dtype=float)
    pred = np.where(pred > 0.5, 1, 0)
    score = metrics.f1_score(label, pred, average='micro')
    return score

def group_kappa_b(df): 
    label = np.array(df.Class_b.tolist(), dtype=int)
    pred = np.array(df.Pred_b.tolist(), dtype=float)
    pred = np.where(pred > 0.5, 1, 0)
    score = metrics.cohen_kappa_score(label, pred)
    return score

def group_acc_b(df): 
    label = np.array(df.Class_b.tolist(), dtype=int)
    pred = np.array(df.Pred_b.tolist(), dtype=float)
    pred = np.where(pred > 0.5, 1, 0)
    score = metrics.accuracy_score(label, pred)
    return score

def group_auc_b(df): 
    label = np.array(df.Class_b.tolist(), dtype=int)
    pred = np.array(df.Pred_b.tolist(), dtype=float)
    score = metrics.roc_auc_score(label, pred)
    return score

In [ ]:
from scipy.special import softmax

def binary_cls(cls):
    if cls == 0 or cls == 1:
        return 0
    elif cls == 2 or cls == 3:
        return 1
    else:
        return None
    
def binary_pred(pred):
    pred = np.array(pred.split(','), dtype=float)
    pred_b = np.array([pred[0]+pred[1], pred[2]+pred[3]])
    pred_b = softmax(pred_b)
    return pred_b[1] # the probability to greater label

## 3DMolCSP-TL

In [ ]:
dfs = []
for mb in range(18): 
    for i in range(5):
        dfs.append(pd.read_csv('../results1205/molnet_chirality_cls_etkdg_csp{}-5fold_tl_{}.csv'.format(str(mb), str(i)), 
                               sep='\t', index_col=0))
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df

In [ ]:
df = df.rename(columns={'SMILES': 'SMILES_iso'})
df['SMILES'] = df['SMILES_iso'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x), isomericSmiles=False))

df['MB'] = df['MB'].astype(int)
df['Class_b'] = df['Class'].apply(binary_cls)
df['Pred_b'] = df['Pred'].apply(binary_pred)
df['Chir_tag'] = df['SMILES_iso'].apply(lambda x: ''.join(['0' if str(atom.GetChiralTag()) == 'CHI_UNSPECIFIED' else '1' for atom in Chem.MolFromSmiles(x).GetAtoms()]))

In [ ]:
df

In [ ]:
df_uniq = df.sort_values(['SMILES_iso', 'Chir_tag', 'MB', 'Class'], ascending=False).drop_duplicates(['SMILES_iso', 'Chir_tag', 'MB'], keep='first').sort_index()

df_uniq = df_uniq.groupby(by=['SMILES', 'Chir_tag', 'MB']).filter(lambda x: len(x) == 2)

In [ ]:
df_pred = df_uniq.groupby(by=['SMILES', 'Chir_tag', 'MB'])['Pred_b'].mean().to_frame(name='Pred_b').reset_index()

df_pred = df_pred.merge(df[['SMILES', 'MB', 'Chir_tag', 'Class_b']], on=['SMILES', 'MB', 'Chir_tag'], how='left').drop_duplicates(['SMILES', 'MB', 'Chir_tag'], keep='first').sort_index()

In [ ]:
df_pred

In [ ]:
auc = df_pred.groupby('MB').apply(group_auc_b)
acc = df_pred.groupby('MB').apply(group_acc_b)
kappa = df_pred.groupby('MB').apply(group_kappa_b)
f1 = df_pred.groupby('MB').apply(group_f1_b)

In [ ]:
print('AUC:', '\n'+'\n'.join(auc.astype(str).tolist()), '\n')
print('ACC:', '\n'+'\n'.join(acc.astype(str).tolist()), '\n')
print('KAPPA:', '\n'+'\n'.join(kappa.astype(str).tolist()), '\n')
print('F1:', '\n'+'\n'.join(f1.astype(str).tolist()), '\n')

### average on k-folds

In [ ]:
dfs = []
for mb in range(18): 
    for i in range(5):
        df_tmp = pd.read_csv('../results0804/molnet_chirality_cls_etkdg_csp{}-5fold_tl_{}.csv'.format(str(mb), str(i)), 
                               sep='\t', index_col=0)
        df_tmp['k-fold'] = str(i)
        dfs.append(df_tmp)
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df = df.rename(columns={'SMILES': 'SMILES_iso'})
df['SMILES'] = df['SMILES_iso'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x), isomericSmiles=False))

df['MB'] = df['MB'].astype(int)
df['Class_b'] = df['Class'].apply(binary_cls)
df['Pred_b'] = df['Pred'].apply(binary_pred)
df['Chir_tag'] = df['SMILES_iso'].apply(lambda x: ''.join(['0' if str(atom.GetChiralTag()) == 'CHI_UNSPECIFIED' else '1' for atom in Chem.MolFromSmiles(x).GetAtoms()]))

In [ ]:
df_uniq = df.sort_values(['SMILES_iso', 'Chir_tag', 'MB', 'Class'], ascending=False).drop_duplicates(['SMILES_iso', 'Chir_tag', 'MB'], keep='first').sort_index()

df_uniq = df_uniq.groupby(by=['SMILES', 'Chir_tag', 'MB', 'k-fold']).filter(lambda x: len(x) == 2)

In [ ]:
df_pred = df_uniq.groupby(by=['SMILES', 'Chir_tag', 'MB', 'k-fold'])['Pred_b'].mean().to_frame(name='Pred_b').reset_index()

df_pred = df_pred.merge(df[['SMILES', 'MB', 'Chir_tag', 
                            'Class_b', 'k-fold']], 
                        on=['SMILES', 'MB', 'Chir_tag', 'k-fold'], 
                        how='left').drop_duplicates(['SMILES', 'MB', 'Chir_tag', 'k-fold'], keep='first').sort_index()

In [ ]:
df_pred

In [ ]:
df_auc = df_pred.groupby(by=['MB', 'k-fold']).apply(group_auc_b).reset_index().rename(columns={0: 'AUC'})

auc_mean = df_auc.groupby('MB').mean().values.tolist()
auc_std = df_auc.groupby('MB').std().values.tolist()

In [ ]:
df_auc[df_auc['MB'] == 6]

In [ ]:
df_auc[df_auc['MB'] == 8]

In [ ]:
df_auc[df_auc['MB'] == 9]

In [ ]:
df_auc[df_auc['MB'] == 11]

In [ ]:
df_auc[df_auc['MB'] == 15]

In [ ]:
print('\n'.join([str(i[0]) for i in auc_mean]))

In [ ]:
print('\n'.join([str(i[0]) for i in auc_std]))

In [ ]:
df_kappa = df_pred.groupby(by=['MB', 'k-fold']).apply(group_kappa_b).reset_index().rename(columns={0: 'KAPPA'})

kappa_mean = df_kappa.groupby('MB').mean().values.tolist()
kappa_std = df_kappa.groupby('MB').std().values.tolist()

In [ ]:
print('\n'.join([str(i[0]) for i in kappa_mean]))

In [ ]:
print('\n'.join([str(i[0]) for i in kappa_std]))

In [ ]:
df_f1 = df_pred.groupby(by=['MB', 'k-fold']).apply(group_f1_b).reset_index().rename(columns={0: 'F1'})

f1_mean = df_f1.groupby('MB').mean().values.tolist()
f1_std = df_f1.groupby('MB').std().values.tolist()

In [ ]:
print('\n'.join([str(i[0]) for i in f1_mean]))

In [ ]:
print('\n'.join([str(i[0]) for i in f1_std]))